# Deep Neural Network

In [ ]:
import os
import sys
sys.path.append(os.path.abspath("../scripts"))
from data_loader import DataLoader
import torch
from torch import optim
import torch.nn as nn
from neural_network.model import NeuralNet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
data_loader = DataLoader()

In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek


def prepare_data_with_resampling(
    X_train, y_train, X_val, y_val, X_test, y_test, resampling_method=None
):
    
    # Load data
    X_train, y_train = data_loader.training_data
    X_val, y_val = data_loader.validation_data
    X_test, y_test = data_loader.test_data

    # Ensure data is converted to NumPy arrays
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_val, y_val = np.array(X_val), np.array(y_val)
    X_test, y_test = np.array(X_test), np.array(y_test)

    # Apply resampling if a method is specified
    if resampling_method == "random_oversample":
        ros = RandomOverSampler(random_state=42)
        X_train, y_train = ros.fit_resample(X_train, y_train)
    elif resampling_method == "random_undersample":
        rus = RandomUnderSampler(random_state=42)
        X_train, y_train = rus.fit_resample(X_train, y_train)
    elif resampling_method == "smote":
        smote = SMOTE(random_state=42)
        X_train, y_train = smote.fit_resample(X_train, y_train)
    elif resampling_method == "smote_tomek":
        smote_tomek = SMOTETomek(random_state=42)
        X_train, y_train = smote_tomek.fit_resample(X_train, y_train)
    elif resampling_method is not None:
        raise ValueError(f"Unsupported resampling method: {resampling_method}")

    # Convert data to PyTorch tensors
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(
        y_train, dtype=torch.long
    )
    X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(
        y_val, dtype=torch.long
    )
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(
        y_test, dtype=torch.long
    )

    # Create DataLoaders
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    val_dataset = TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader, val_loader, test_loader

In [4]:
# Training function
def train_model(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).float()
            optimizer.zero_grad()
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        val_loss, val_metrics = evaluate_model(model, val_loader)
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f} | Val Metrics: {val_metrics}")

In [5]:
# Evaluation function
def evaluate_model(model, loader):
    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).float()
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            preds = (outputs > 0.5).long()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    metrics = {
        "Accuracy": accuracy_score(all_labels, all_preds),
        "Precision": precision_score(all_labels, all_preds),
        "Recall": recall_score(all_labels, all_preds),
        "F1 Score": f1_score(all_labels, all_preds)
    }
    return val_loss / len(loader), metrics

In [ ]:
# Specify the resampling method for this run
resampling_method = "smote"  # Options: None, "random_oversample", "random_undersample", "smote", "smote_tomek"

# Prepare DataLoaders with the chosen resampling method
train_loader, val_loader, test_loader = prepare_data_with_resampling(
    X_train, y_train, X_val, y_val, X_test, y_test, resampling_method=resampling_method
)

# Define model, loss, and optimizer
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
input_size = X_train.shape[1]
model = NeuralNet(input_size).to(device)
print(model)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using mps device


In [8]:
# Train the model
train_model(model, train_loader, val_loader, epochs=20)

Epoch 1/20
Train Loss: 0.5189 | Val Loss: 0.5249 | Val Metrics: {'Accuracy': 0.7161739130434782, 'Precision': np.float64(0.3283942833123161), 'Recall': np.float64(0.7663070132417852), 'F1 Score': np.float64(0.4597616595556863)}
Epoch 2/20
Train Loss: 0.5050 | Val Loss: 0.4890 | Val Metrics: {'Accuracy': 0.7308212560386473, 'Precision': np.float64(0.3383719628261113), 'Recall': np.float64(0.7410495340853359), 'F1 Score': np.float64(0.4646014297793835)}
Epoch 3/20
Train Loss: 0.4978 | Val Loss: 0.5173 | Val Metrics: {'Accuracy': 0.7210434782608696, 'Precision': np.float64(0.33114648311464834), 'Recall': np.float64(0.7550269740068661), 'F1 Score': np.float64(0.4603767942583732)}
Epoch 4/20
Train Loss: 0.4916 | Val Loss: 0.5050 | Val Metrics: {'Accuracy': 0.7353429951690821, 'Precision': np.float64(0.3422191843244475), 'Recall': np.float64(0.7366356056890633), 'F1 Score': np.float64(0.4673304293714997)}
Epoch 5/20
Train Loss: 0.4861 | Val Loss: 0.4795 | Val Metrics: {'Accuracy': 0.75161352

In [9]:
# Save the trained model
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_save_path = f'../models/neural_net_{timestamp}.pkl'

torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to ../models/neural_net_20241129_222423.pkl


In [ ]:
import torch
from neural_network.model import NeuralNet

# Define the input size (must match the saved model)
input_size = 21  # Number of features

# Instantiate the model and load state dict
model = NeuralNet(input_size)
model.load_state_dict(torch.load(model_save_path))
model.eval()  # Set to evaluation mode

# Example usage
X_val_data, _ = data_loader.validation_data
X_val_data = torch.tensor(X_val_data.values, dtype=torch.float32)

with torch.no_grad():
    predictions = (model(X_val_data).squeeze() > 0.5).long()
    print("Predictions:", predictions)

# TODO: check if it makes sense to to make prediciotns like this and use 0.5 threshold

Predictions: tensor([0, 1, 0,  ..., 1, 1, 0])


/var/folders/mx/80hdp5rj4v550swmwk6ryqmc0000gn/T/ipykernel_70519/1837935747.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_p

In [ ]:

# Load data
X_train, y_train = data_loader.training_data
X_val, y_val = data_loader.validation_data
X_test, y_test = data_loader.test_data


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
report = classification_report(y_val, predictions, digits=4, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0     0.9270    0.7713    0.8420     21797
         1.0     0.3559    0.6756    0.4662      4078

    accuracy                         0.7562     25875
   macro avg     0.6415    0.7234    0.6541     25875
weighted avg     0.8370    0.7562    0.7828     25875

